In [2]:
from Dental_Tool.Data_processing import *
from Dental_Tool.Dental_Model import *
from Dental_Tool.Process_results import *
from Dental_Tool.Dataloader import *
from Dental_Tool.KFold_v3 import *

In [4]:
directory = [ 
                "Dental_Data/PBL/10_20200901", 
                "Dental_Data/PBL/10_20200901_Flip", 
                "Dental_Data/PBL/10_clahe_20200901", 
                "Dental_Data/PBL/10_clahe_20200901_Flip"
            ]

directory = [ i + "/mapping.json" for i in directory]
argscale_num = len(directory) * 20
data = load_json(directory, interdental=False)
dataset = json_2_dataframe_PBL(data, mode="molar")
dataset

,Path,State,Class,source,tooth_num,angle
0,Dental_Data/PBL/10_20200901/15-53-19-666_00040...,3,2,NN_191024_151631_BE78BA_2,2,-10
1,Dental_Data/PBL/10_20200901_Flip/15-53-19-666_...,3,2,NN_191024_151631_BE78BA_2,2,-10
2,Dental_Data/PBL/10_clahe_20200901/16-31-09-808...,3,2,NN_191024_151631_BE78BA_2,2,-10
3,Dental_Data/PBL/10_clahe_20200901_Flip/16-31-0...,3,2,NN_191024_151631_BE78BA_2,2,-10
4,Dental_Data/PBL/10_20200901/15-53-19-697_00040...,3,2,NN_191024_151631_BE78BA_2,2,-9
...,...,...,...,...,...,...
106475,Dental_Data/PBL/10_clahe_20200901_Flip/17-03-1...,1,0,NN_140521_095055_C0A4EA_19,19,8
106476,Dental_Data/PBL/10_20200901/16-30-47-021_S4210...,1,0,NN_140521_095055_C0A4EA_19,19,9
106477,Dental_Data/PBL/10_20200901_Flip/16-30-47-021_...,1,0,NN_140521_095055_C0A4EA_19,19,9
106478,Dental_Data/PBL/10_clahe_20200901/17-03-16-474...,1,0,NN_140521_095055_C0A4EA_19,19,9


In [17]:
image_size = 256
input_shape = (image_size, image_size, 1)
batch_size = 100
kernel_size = 3
filters = 16
latent_dim = 128 # 隐变量取2维只是为了方便后面画图
epochs = 30
num_classes = classes = 3

x_in = Input(shape=input_shape)
x = x_in

deep_layer = 3

for i in range(deep_layer):
    filters *= 2
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               activation='relu',
               strides=2,
               padding='same')(x)

# 备份当前shape，等下构建decoder的时候要用
shape = K.int_shape(x)

x = Flatten()(x)
x = Dense(latent_dim, activation='relu')(x)
# # 算p(Z|X)的均值和方差
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

y = Input(shape=(num_classes,)) # 输入类别
yh = Dense(latent_dim)(y) # 这里就是直接构建每个类别的均值



# 重参数技巧
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=K.shape(z_mean))
    return z_mean + K.exp(z_log_var / 2) * epsilon

# 重参数层，相当于给输入加入噪声
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# 解码层，也就是生成器部分
# 先搭建为一个独立的模型，然后再调用模型
latent_inputs = Input(shape=(latent_dim,))
# label_inputs = Input(shape=(num_classes,))
# x = Concatenate()([latent_inputs])
x = Dense(shape[1] * shape[2] * shape[3], activation='relu')(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3]))(x)

for i in range(deep_layer):
    x = Conv2DTranspose(filters=filters,
                        kernel_size=kernel_size,
                        activation='relu',
                        strides=2,
                        padding='same')(x)
    filters //= 2

outputs = Conv2DTranspose(filters=1,
                          kernel_size=kernel_size,
                          activation='sigmoid',
                          padding='same')(x)

# 搭建为一个独立的模型
decoder = Model(latent_inputs, outputs)

x_out = decoder(z)

# 建立模型
vae = Model([x_in, y] , [x_out, yh])

# xent_loss是重构loss，kl_loss是KL loss
xent_loss = K.sum(K.binary_crossentropy(x_in, x_out), axis=-1)

# 只需要修改K.square(z_mean)为K.square(z_mean - yh)，也就是让隐变量向类内均值看齐
kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean - yh) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(xent_loss + kl_loss)

# # xent_loss是重构loss，kl_loss是KL loss
# xent_loss = K.sum(K.binary_crossentropy(x_in, x_out), axis=[1, 2, 3])
# kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
# vae_loss = K.mean(xent_loss + kl_loss)

# add_loss是新增的方法，用于更灵活地添加各种loss
vae.add_loss(vae_loss)
vae.compile(optimizer='rmsprop')
vae.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 128, 128, 32) 320         input_10[0][0]                   
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 64, 64, 64)   18496       conv2d_10[0][0]                  
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 32, 32, 128)  73856       conv2d_11[0][0]                  
____________________________________________________________________________________________

C:\Users\Lab620\Anaconda3\envs\lawrence\lib\site-packages\keras\engine\training_utils.py:819: UserWarning: Output model_7 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to model_7.
  'be expecting any data to be passed to {0}.'.format(name))
C:\Users\Lab620\Anaconda3\envs\lawrence\lib\site-packages\keras\engine\training_utils.py:819: UserWarning: Output dense_19 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to dense_19.
  'be expecting any data to be passed to {0}.'.format(name))


In [10]:
def sampling(args: tuple):
        # we grab the variables from the tuple
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.,
                                  stddev=epsilon_std)
        return z_mean + K.exp(z_log_var / 2) * epsilon

In [14]:
def load_images(path_list, resize):
        X = []
        for path in tqdm(path_list):
                image = cv2.imread(path, 0)
                image = cv2.resize(image, resize)
                image = image.astype("float32") / 255.0
#                 image = image - np.mean(image)
                image = np.expand_dims(image, axis=2)
                X.append(image)
        return np.array(X)


for train, valid, test, train_gen, valid_gen, test_gen in \
    K_Fold_balance_data_generator(dataset, argscale_num, batch_size=32, k_fold_num=5):
        
        x_train, y_train = load_images(train["Path"], (image_size, image_size)), to_categorical(train["Class"], classes)
        x_valid, y_valid = load_images(valid["Path"], (image_size, image_size)), to_categorical(valid["Class"], classes)
        x_test, y_test = load_images(test["Path"], (image_size, image_size)), to_categorical(test["Class"], classes)
        
        print(x_train.shape), print(y_train.shape)
        break

stage 0: 10400, stage 1: 27120, stage 2: 17360, stage 3: 8640
stage 0: 3600, stage 1: 9120, stage 2: 5680, stage 3: 2880
stage 0: 3520, stage 1: 9200, stage 2: 5840, stage 3: 3120



(25920, 256, 256, 1)
(25920, 3)


In [18]:
vae.fit([x_train, y_train], 
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=([x_valid, y_valid], None))

Train on 25920 samples, validate on 8640 samples
Epoch 1/30


InternalError:  Blas GEMM launch failed : a.shape=(100, 3), b.shape=(3, 128), m=100, n=128, k=3
	 [[node dense_19/MatMul (defined at C:\Users\Lab620\Anaconda3\envs\lawrence\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_keras_scratch_graph_4972]

Function call stack:
keras_scratch_graph
